In [363]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import os
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_breusch_godfrey, normal_ad
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import shapiro, ks_2samp, chi2, f
import pickle
from datetime import datetime


def Log_diff(df):
    df_diff = np.log(df/df.shift(1))
    df_diff = df_diff.dropna()
    return df_diff

btc_prices = pd.read_csv("btc-usd-max.csv",index_col=0,parse_dates=True)
btc_prices.rename(columns = {'price':"BTC_close"},inplace=True)
btc_prices.index = btc_prices.index.tz_localize(None)
eth_prices = pd.read_csv("eth-usd-max.csv",index_col=0,parse_dates=True)
eth_prices.rename(columns = {'price':"ETH_close"},inplace=True)
eth_prices.index = eth_prices.index.tz_localize(None)
sol_prices = pd.read_csv("sol-usd-max.csv",index_col=0,parse_dates=True)
sol_prices.rename(columns = {'price':"SOL_close","market_cap":"SOL_mc","total_volume":"SOL_volume"},inplace=True)
sol_prices.index = sol_prices.index.tz_localize(None)

funding_sol = pd.read_excel("sol_funding.xlsx",index_col=0,parse_dates=True)
funding_sol.rename(columns={"daily_funding_rate":"SOL_fr"},inplace=True)

sol_onchain = pd.read_excel("SOL_onchain_data.xlsx",sheet_name= "sol_epoch_3_data",index_col=0,parse_dates=True)
sol_tv = pd.read_csv("sol_tvl.csv",index_col=0,parse_dates=True)

df = pd.merge(sol_prices, pd.DataFrame(btc_prices["BTC_close"]), left_index=True, right_index=True)
df = pd.merge(df, eth_prices["ETH_close"],left_index=True,right_index=True)
df = pd.merge(df, funding_sol["SOL_fr"],right_index=True,left_index=True)
df = pd.merge(df, sol_tv["TVL_USD"],right_index=True,left_index=True)
sol_onchain_aligned = sol_onchain[["apy block", "apy jito","network_active_stake"]].reindex(df.index)

sol_onchain

,nb_epoch_year,network_active_stake,network_active_stake_jito,network_prio_fees,epoch_prio_rate,network_jito_fees,epoch_jito_rate,apy block,apy jito
end_date,,,,,,,,,
2025-05-18,184.794842,389388940.0,3.829364e+08,16972.32103,0.000044,24721.212857,0.000065,0.008087,0.012001
2025-05-16,184.470775,390142087.0,3.814012e+08,25418.66868,0.000065,42824.762428,0.000112,0.012091,0.020928
2025-05-14,184.136024,388734676.0,3.748948e+08,30187.80478,0.000078,48005.241018,0.000128,0.014402,0.023857
2025-05-12,185.177622,372882925.0,3.729236e+08,15678.26534,0.000042,24255.551705,0.000065,0.007816,0.012117
2025-05-10,185.245016,371121635.0,4.023168e+08,16794.70374,0.000045,27073.007635,0.000067,0.008418,0.012543
...,...,...,...,...,...,...,...,...,...
2020-02-18,134.520064,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
2020-02-15,134.519491,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
2020-02-13,134.520064,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000


In [364]:

df = pd.concat([df, sol_onchain_aligned], axis=1)
df = df.ffill().dropna()
df


,SOL_close,SOL_mc,SOL_volume,BTC_close,ETH_close,SOL_fr,TVL_USD,apy block,apy jito,network_active_stake
2021-03-20,14.182867,3.784531e+09,8.684253e+07,58243.270609,1817.132963,0.000499,147690914,0.000000,0.000000,2.460124e+08
2021-03-21,14.275804,3.827195e+09,8.634340e+07,58376.158471,1817.860143,0.000327,151935325,0.000000,0.000000,2.460124e+08
2021-03-22,14.306230,3.820744e+09,7.948416e+07,57573.539345,1790.378075,0.000128,152981122,0.000000,0.000000,2.460124e+08
2021-03-23,14.870337,3.996884e+09,3.265923e+08,54370.137558,1686.891197,0.000274,162065403,0.000000,0.000000,2.473500e+08
2021-03-24,14.194568,3.797038e+09,1.338872e+08,54584.869947,1673.859184,0.000300,156101311,0.000000,0.000000,2.473500e+08
...,...,...,...,...,...,...,...,...,...,...
2025-05-15,176.455072,9.167281e+10,5.807896e+09,103594.425751,2603.718089,0.000020,9377206736,0.014402,0.023857,3.887347e+08
2025-05-16,168.752697,8.770307e+10,5.419257e+09,103708.851364,2542.296692,0.000062,9133860717,0.012091,0.020928,3.901421e+08
2025-05-17,167.768124,8.732407e+10,4.389550e+09,103556.034940,2544.387038,-0.000002,9023656924,0.012091,0.020928,3.901421e+08
2025-05-18,165.939417,8.625805e+10,3.269430e+09,103212.364839,2475.054393,0.000149,9010591266,0.008087,0.012001,3.893889e+08


In [365]:
df_macro = pd.read_excel("MacroFactorsUS.xlsx",index_col=0,parse_dates=True,skiprows=range(1,2))
data_macro = df_macro.ffill().dropna()

data_macro



,Equity Factor,Mkt Vol Factor,ECSURPUS Index,Econ Surprise Factor,US 10Y Rate Level Factor,US 2Y Rate Level Factor,US 3M Rate Level Factor,Tsy Bond Factor,DXY Factor,FEDL01 Index,Commodity Factor,CPI YOY Index,Inflation Factor
2020-01-31,3225.52,18.84,0.035209,7.6,1.506796,1.313186,1.538229,2427.69,97.390010,1.59,76.3521,2.48657,363.9027
2020-02-01,3225.52,18.84,0.035209,7.6,1.506796,1.313186,1.538229,2427.69,97.390010,1.59,76.3521,2.48657,363.9027
2020-02-02,3225.52,18.84,0.035209,7.6,1.506796,1.313186,1.538229,2427.69,97.390010,1.59,76.3521,2.48657,363.9027
2020-02-03,3248.92,17.97,0.067839,15.9,1.527200,1.353006,1.553525,2428.45,97.800000,1.59,74.7039,2.48657,362.1421
2020-02-04,3297.59,16.05,0.089131,19.4,1.599128,1.408987,1.546237,2415.67,97.961010,1.59,74.6740,2.48657,360.6754
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-16,5958.38,17.24,-0.092805,-12.6,4.476962,3.999450,4.335632,2337.49,101.092000,4.33,104.2918,2.31129,337.1940
2025-05-17,5958.38,17.24,-0.092805,-12.6,4.476962,3.999450,4.335632,2337.49,101.092000,4.33,104.2918,2.31129,337.1940
2025-05-18,5958.38,17.24,-0.092805,-12.6,4.476962,3.999450,4.335632,2337.49,101.092000,4.33,104.2918,2.31129,337.1940
2025-05-19,5963.60,18.14,-0.090864,-10.6,4.447354,3.976511,4.337704,2337.18,100.426000,4.33,102.9526,2.31129,338.4940


In [366]:
data_macro['Risk-on/Risk-off Factor'] = data_macro['Equity Factor'] - data_macro['Tsy Bond Factor']

data_macro['Nominal Rates Factor'] = data_macro['US 10Y Rate Level Factor'].diff()

data_macro['FEDL01 Change Factor'] = data_macro['FEDL01 Index'].diff() 

data_macro['Curve Shape'] = data_macro['US 10Y Rate Level Factor'].diff()-data_macro['US 2Y Rate Level Factor'].diff()



data_macro['Equity Weight'] = 1/data_macro['Equity Factor'].rolling(window=36, min_periods=24).std()
data_macro['Tsy Bond Weight'] = 1/data_macro['Tsy Bond Factor'].rolling(window=36, min_periods=24).std()
data_macro['Equity Weight'] = data_macro['Equity Weight']/(data_macro['Equity Weight'] + data_macro['Tsy Bond Weight'])
data_macro['Tsy Bond Weight'] = 1 - data_macro['Equity Weight']

data_macro['Risk Parity Factor'] = (data_macro['Equity Weight'] * data_macro['Equity Factor']) + (data_macro['Tsy Bond Weight'] * data_macro['Tsy Bond Factor'])


data_to_normalize = pd.merge(df,data_macro, left_index=True,right_index=True)

data_to_normalize

,SOL_close,SOL_mc,SOL_volume,BTC_close,ETH_close,SOL_fr,TVL_USD,apy block,apy jito,network_active_stake,...,Commodity Factor,CPI YOY Index,Inflation Factor,Risk-on/Risk-off Factor,Nominal Rates Factor,FEDL01 Change Factor,Curve Shape,Equity Weight,Tsy Bond Weight,Risk Parity Factor
2021-03-20,14.182867,3.784531e+09,8.684253e+07,58243.270609,1817.132963,0.000499,147690914,0.000000,0.000000,2.460124e+08,...,95.4202,1.67621,388.4664,1464.80,0.000000,0.0,0.000000,0.279399,0.720601,2857.563946
2021-03-21,14.275804,3.827195e+09,8.634340e+07,58376.158471,1817.860143,0.000327,151935325,0.000000,0.000000,2.460124e+08,...,95.4202,1.67621,388.4664,1464.80,0.000000,0.0,0.000000,0.276883,0.723117,2853.878872
2021-03-22,14.306230,3.820744e+09,7.948416e+07,57573.539345,1790.378075,0.000128,152981122,0.000000,0.000000,2.460124e+08,...,95.6514,1.67621,389.4017,1485.36,-0.026452,0.0,-0.024466,0.268093,0.731907,2853.443897
2021-03-23,14.870337,3.996884e+09,3.265923e+08,54370.137558,1686.891197,0.000274,162065403,0.000000,0.000000,2.473500e+08,...,93.4362,1.67621,389.8500,1447.87,-0.073997,0.0,-0.072005,0.257353,0.742647,2835.263646
2021-03-24,14.194568,3.797038e+09,1.338872e+08,54584.869947,1673.859184,0.000300,156101311,0.000000,0.000000,2.473500e+08,...,95.0404,1.67621,390.6712,1423.36,-0.012177,0.0,-0.013477,0.251612,0.748388,2823.914684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-15,176.455072,9.167281e+10,5.807896e+09,103594.425751,2603.718089,0.000020,9377206736,0.014402,0.023857,3.887347e+08,...,104.6105,2.31129,336.8476,3579.93,-0.104818,0.0,-0.014802,0.056309,0.943691,2538.581990
2025-05-16,168.752697,8.770307e+10,5.419257e+09,103708.851364,2542.296692,0.000062,9133860717,0.012091,0.020928,3.901421e+08,...,104.2918,2.31129,337.1940,3620.89,0.045479,0.0,0.006801,0.053825,0.946175,2532.383800
2025-05-17,167.768124,8.732407e+10,4.389550e+09,103556.034940,2544.387038,-0.000002,9023656924,0.012091,0.020928,3.901421e+08,...,104.2918,2.31129,337.1940,3620.89,0.000000,0.0,0.000000,0.050262,0.949738,2519.483848
2025-05-18,165.939417,8.625805e+10,3.269430e+09,103212.364839,2475.054393,0.000149,9010591266,0.008087,0.012001,3.893889e+08,...,104.2918,2.31129,337.1940,3620.89,0.000000,0.0,0.000000,0.047195,0.952805,2508.378766


In [367]:
zsc_prd = 30

data_to_normalize = data_to_normalize.drop(columns={"SOL_close","BTC_close","ETH_close","Equity Factor","network_active_stake"})

data_normalized = (data_to_normalize - data_to_normalize.rolling(window=zsc_prd, min_periods=min(zsc_prd,24)).mean()) / \
            data_to_normalize.rolling(window=zsc_prd, min_periods=min(zsc_prd,24)).std()

data_normalized_shifted = data_normalized.shift(30)

data_normalized_shifted = data_normalized_shifted.ffill().dropna()

data_normalized = data_normalized.ffill().dropna()


In [368]:
df = pd.merge(df,data_macro["Equity Factor"],right_index=True,left_index=True)

data_merged_shifted = pd.merge(df[["SOL_close","BTC_close","ETH_close","Equity Factor","network_active_stake"]].shift(-1),data_normalized_shifted, left_index=True,right_index=True)
data_merged = pd.merge(df[["SOL_close","BTC_close","ETH_close","Equity Factor","network_active_stake"]].shift(-1),data_normalized, left_index=True,right_index=True)

data_merged.dropna(inplace=True)
data_merged_shifted.dropna(inplace=True)

In [369]:
data_merged["rSOL"] = Log_diff(data_merged["SOL_close"])
data_merged["rETH"] = Log_diff(data_merged["ETH_close"])
data_merged["rBTC"] = Log_diff(data_merged["BTC_close"])
data_merged["rnetwork_active_stake"] = Log_diff(data_merged["network_active_stake"])
data_merged["Equity Factor"] = Log_diff(data_merged["Equity Factor"])

data_merged.drop(["SOL_close","ETH_close","BTC_close"],axis = 1,inplace=True)

final_data = data_merged.ffill().dropna()


data_merged_shifted["rSOL"] = Log_diff(data_merged_shifted["SOL_close"])
data_merged_shifted["rETH"] = Log_diff(data_merged_shifted["ETH_close"])
data_merged_shifted["rBTC"] = Log_diff(data_merged_shifted["BTC_close"])
data_merged_shifted["rnetwork_active_stake"] = Log_diff(data_merged_shifted["network_active_stake"])
data_merged_shifted["Equity Factor"] = Log_diff(data_merged_shifted["Equity Factor"])

data_merged_shifted.drop(["SOL_close","ETH_close","BTC_close"],axis = 1,inplace=True)

final_data_shifted = data_merged_shifted.ffill().dropna()



final_data

,Equity Factor,network_active_stake,SOL_mc,SOL_volume,SOL_fr,TVL_USD,apy block,apy jito,Mkt Vol Factor,ECSURPUS Index,...,Nominal Rates Factor,FEDL01 Change Factor,Curve Shape,Equity Weight,Tsy Bond Weight,Risk Parity Factor,rSOL,rETH,rBTC,rnetwork_active_stake
2022-12-15,-0.011200,376161006.0,0.957322,0.819833,0.883365,-0.108512,-0.561054,3.678768,0.683808,-1.368939,...,-0.334658,5.294651,-1.637280,2.337316,-2.337316,1.978037,-0.010992,-0.034724,-0.026767,0.000649
2022-12-16,0.000000,376161006.0,0.746700,-0.282060,0.800185,-0.029683,0.009898,2.782767,0.602020,-1.273049,...,0.717212,-0.182574,3.020347,1.790003,-1.790003,1.396458,-0.131320,-0.080524,-0.042088,0.000000
2022-12-17,0.000000,367296562.0,-1.888572,0.321945,0.252141,-3.232688,0.038411,2.394137,0.651976,-1.187855,...,0.163950,-0.182574,0.010366,1.359884,-1.359884,1.053000,0.020450,0.019916,0.010932,-0.023848
2022-12-18,-0.009048,367296562.0,-1.365256,-0.489778,0.093588,-3.005429,1.139328,0.630920,0.670035,-1.117382,...,0.206241,-0.182574,-0.008663,1.003488,-1.003488,0.753701,-0.008759,-0.002230,-0.003796,0.000000
2022-12-19,0.001037,368601282.0,-1.338724,-1.373072,0.112007,-2.727105,1.091946,0.594033,0.554810,-0.919313,...,1.850013,-0.182574,0.762071,0.496074,-0.496074,0.105998,-0.039502,-0.016815,-0.019313,0.003546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-14,0.004124,388734676.0,2.369405,1.150778,1.031686,2.311776,3.670648,3.598574,-1.629027,0.149642,...,1.359868,0.182574,0.632369,-0.916194,0.916194,-0.584123,-0.042162,-0.027622,-0.005680,0.000000
2025-05-15,0.006981,390142087.0,1.716022,0.854337,0.078734,1.819350,2.950530,2.906290,-1.677568,-0.861135,...,-2.084310,0.182574,-0.415451,-1.295406,1.295406,-0.542518,-0.044632,-0.023873,0.001104,0.003614
2025-05-16,0.000000,390142087.0,1.143868,0.599136,0.243312,1.399720,1.681329,1.927535,-1.715190,-2.085651,...,0.758552,0.182574,0.228767,-1.585970,1.585970,-0.742326,-0.005852,0.000822,-0.001475,0.000000
2025-05-17,0.000000,389388940.0,1.029691,0.031362,-0.021616,1.174683,1.563727,1.804467,-1.588585,-1.882848,...,-0.100246,0.182574,0.048410,-1.948126,1.948126,-1.143334,-0.010960,-0.027627,-0.003324,-0.001932


In [370]:

with open('data_v1_offset.pickle', 'wb') as handle: pickle.dump(final_data_shifted, handle)
with open('data_v1.pickle', 'wb') as handle: pickle.dump(final_data, handle)

